In [7]:
import pandas as pd
import numpy as np
from pathlib import Path

sample_loc_lookup = pd.read_csv("helper_data/bldg_to_sampleloc.csv")
sample_locs = sample_loc_lookup["sample_loc_alpha"].unique()

dfs = []

for s in sample_locs:
    bldg_locs = sample_loc_lookup.loc[sample_loc_lookup["sample_loc_alpha"] == s, "building"]

    x = None
    for b in bldg_locs:
        df = pd.read_csv(f"meters_csv/{sample_loc_lookup.loc[sample_loc_lookup['building'] == b, 'filename'].iloc[0]}")
        df = df[df["Good"] == "TRUE"]
        df["timestamp"] = pd.to_datetime(df["Timestamp"]).dt.round("min")
        df["volume"] = df["Value"].astype(float)
        df = df.drop_duplicates(subset="volume")
        df["date"] = df["timestamp"].dt.date
        
        if s in ["PU", "PL"]:
            df["d_volume"] = df["volume"].diff().fillna(0)
            df = df[df["d_volume"] == 1]
        
        df = df.groupby("date")["volume"].agg(["min", "max"])
        df["daily_vol"] = df["max"] - df["min"]
        
        if x is None:
            x = df
        else:
            x["daily_vol"] += df["daily_vol"]

    dfs.append(x.reset_index())

volume_daily = pd.concat(dfs, axis=0).reset_index(drop=True)

# Fix the column names assignment
columns = ["date"] + sample_locs.tolist()
volume_daily.columns = columns

volume_daily = volume_daily.melt(id_vars="date", var_name="sample_loc", value_name="vol_daily")
volume_daily.to_csv("test.csv", index=False)


/tmp/ipykernel_954007/202661679.py:15: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"meters_csv/{sample_loc_lookup.loc[sample_loc_lookup['building'] == b, 'filename'].iloc[0]}")
/tmp/ipykernel_954007/202661679.py:15: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"meters_csv/{sample_loc_lookup.loc[sample_loc_lookup['building'] == b, 'filename'].iloc[0]}")
/tmp/ipykernel_954007/202661679.py:15: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"meters_csv/{sample_loc_lookup.loc[sample_loc_lookup['building'] == b, 'filename'].iloc[0]}")
/tmp/ipykernel_954007/202661679.py:15: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"meters_csv/{sample_loc_lookup.loc[sample_loc_lookup['building'] == b, 'filename']

ValueError: Length mismatch: Expected axis has 4 elements, new values have 16 elements